# Project: Traffic Simulation Model


## Introduction

This project implements a traffic simulation model using the Mesa framework. The simulation aims to model the movement of cars within a city grid, incorporating traffic lights and parking spots. The primary goal is to simulate realistic traffic behavior, including car movements, traffic light interactions, and parking dynamics.

The simulation is built on a 24x24 grid, where each cell can represent different elements such as roads, buildings, parking spots, and traffic lights. Car agents navigate through the grid based on predefined directional coordinates and traffic light states, aiming to reach their target parking spots.

Key features of the project include:

- Car Agents: Simulated cars that move around the grid, following traffic rules and aiming to park in available spots.
- Traffic Light Agents: Traffic lights that control the flow of car agents at intersections, switching between red and green states.
- Grid Representation: A multi-layer grid that includes buildings, roads, and parking spots.
- Visualization: Tools for visualizing the simulation, including the positions of car agents and the states of traffic lights.

The project leverages the Mesa framework for agent-based modeling and includes various components for data collection and visualization to analyze the simulation results.

## Installation



To set up the project, you need to install several dependencies. These libraries are essential for running the traffic simulation model and visualizing the results.

### Required Libraries

- **Mesa (version 3.1.0.dev)**: A Python framework for agent-based modeling. It provides the core functionality for creating and managing agents, as well as tools for visualization and data collection.
- **Seaborn**: A data visualization library based on Matplotlib. It is used for creating informative and attractive statistical graphics.
- **Solara**: A library for building interactive web applications in Python. It is used for creating interactive visualizations of the simulation.
- **Pandas**: A powerful data manipulation and analysis library. It is used for handling and analyzing data within the simulation.
- **NumPy**: A fundamental package for scientific computing with Python. It is used for numerical operations and handling arrays.
- **Random**: A module for generating random numbers. It is used for randomizing agent behaviors and movements.
- **Time**: A module for time-related functions. It is used for managing simulation timing and delays.

### Installation Steps

To install the required libraries, you can use the following commands:
```python
#%pip install --quiet --upgrade mesa==3.1.0.dev
#%pip install solara
#%pip install seaborn
#%pip install pandas
#%pip install numpy

In [2]:

import time
# Data visualization tools.
import seaborn as sns
import random
import numpy as np

# Data manipulation and analysis.
import pandas as pd

#Mesa library
import mesa
from mesa.visualization.utils import update_counter
from mesa.visualization import SolaraViz, make_plot_component, make_space_component
from mesa.datacollection import DataCollector
print(f"Mesa version: {mesa.__version__}")


Mesa version: 3.0.3


## Map Construction



In this section, we will define the structure and layout of the simulation environment. The map construction involves setting up a grid that represents the city, including buildings, parking spots, directional coordinates, and traffic lights. Each of these elements plays a crucial role in the simulation:

- **Buildings**: Represent static structures within the city grid. They serve as obstacles that car agents must navigate around.
- **Parking Spots**: Designated areas where car agents aim to park. These spots are the primary destinations for the car agents.
- **Directional Coordinates**: Predefined paths that guide the movement of car agents through the grid. These coordinates help simulate realistic traffic flow.
- **Traffic Lights**: Control the flow of car agents at intersections. They switch between red and green states to manage traffic and prevent collisions.

We will use the Mesa framework to create and manage the grid, ensuring that all elements are correctly placed and interact as expected. The map construction is a crucial step in creating a realistic and functional traffic simulation model.

#### Buildings

In [3]:
from mapBuild.buildings import buildings_coords

#### Parking spots

In [4]:
from mapBuild.parkingSpots import parking_spots

#### Direccional coords(Left, Right, Down, Up)

In [5]:
from mapBuild.leftCoords import left_coords
from mapBuild.rightCoords import right_coords
from mapBuild.upCoords import up_coords
from mapBuild.downCoords import down_coords

#### Secondary directional coord(DownLeft, DownRight, UpLeft, UpRight)

In [6]:
from mapBuild.downLeftCoords import down_left_coords
from mapBuild.downRightCoords import down_right_coords
from mapBuild.upLeftCoords import up_left_coords
from mapBuild.upRightCoords import up_right_coords

from mapBuild.monitoring_coords import monitoring_coords

#### Traffic Lights

In [7]:
from mapBuild.trafficLights import traffic_light_coords

In [8]:
# Storing the coordinates in a dictionary
coords = {
    "left_coords": left_coords,
    "right_coords": right_coords,
    "up_coords": up_coords,
    "down_coords": down_coords,
    "down_left_coords": down_left_coords,
    "down_right_coords": down_right_coords,
    "up_left_coords": up_left_coords,
    "up_right_coords": up_right_coords,
    "monitoring_coords": monitoring_coords,
}

## Traffic Model

In this section, we will explain the traffic simulation model, its components, and main functionalities. The model is designed to simulate the movement of car agents within a city grid, incorporating traffic lights and parking spots to create a realistic traffic environment.

### Components

1. **Car Agents**:
   - **Purpose**: Simulate individual cars moving through the city grid.
   - **Behavior**: Car agents navigate the grid based on directional coordinates, obey traffic lights, and aim to park in available parking spots.

2. **Traffic Light Agents**:
   - **Purpose**: Control the flow of car agents at intersections.
   - **Behavior**: Traffic lights switch between red and green states at regular intervals, managing the movement of car agents and preventing collisions.

3. **Grid**:
   - **Purpose**: Represent the city layout, including roads, buildings, parking spots, and traffic lights.
   - **Structure**: A 24x24 grid where each cell can represent different elements such as roads, buildings, parking spots, and traffic lights.

### Main Functionalities

1. **Initialization**:
   - The model initializes the grid, placing buildings, parking spots, directional coordinates, and traffic lights in their respective positions.
   - Car agents are created and placed on the grid, ready to navigate the city.

2. **Agent Movement**:
   - Car agents move through the grid based on predefined directional coordinates.
   - They obey traffic lights and navigate around buildings to reach their target parking spots.

3. **Traffic Light Control**:
   - Traffic light agents manage the flow of car agents at intersections.
   - They switch between red and green states at regular intervals, ensuring smooth traffic flow and preventing collisions.

4. **Parking Dynamics**:
   - Car agents aim to park in available parking spots.
   - The model tracks the occupancy of parking spots and updates the status of car agents accordingly.

This comprehensive model allows for the simulation of realistic traffic behavior, providing valuable insights into traffic dynamics and potential improvements for urban planning.

In [ ]:
class TrafficModel(mesa.Model):
    def __init__(
        self,
        width=24,
        height=24,
        num_agents=1,
        coords = coords,
        buildings_coords=None,
        parking_coords=None,
        traffic_light_coords=None,
    ):
        
        super().__init__()
        #ATTRIBUTES OF THE MODEL----------------------------------------------------------------------------------
        self.random = random.Random()
        self.steps = 0

        #Parameters
        self.width = width
        self.height = height
        self.num_agents = num_agents
        self.coords = coords
        self.buildings_coords = buildings_coords
        self.parkings_coords = parking_coords
        self.traffic_light_coords = traffic_light_coords

        # Dictionary to store directions for each cell
        self.directions = {}

        # Global map to store the positions of all agents at each step
        self.global_map = {}
        
        
        #Create a dictionary mapping each parking spot to a unique key starting from 1
        self.ParkingSpots = {i + 1: spot for i, spot in enumerate(parking_coords)}
        
        # Initialize grid layers
        self.initialize_layers()
    
        #CREATION OF AGENTS IN THE GRID---------------------------------------    
        # Initialize the allowed directions for each cell
        self.initialize_directions(self.coords)

        # Create the CarAgents and place them on the grid
        self.create_CarAgents()
        self.create_CarAgents_no_target()

        # Place the traffic lights on the grid
        self.place_TrafficLight_agents()

        # Initialize the DataCollector
        self.datacollector = mesa.DataCollector(
            model_reporters={},
            agent_reporters={
                "TargetParkingSpot": lambda agent: (
                    agent.target_parking_spot if isinstance(agent, CarAgent) else None
                )
            },
        )

        # Collect initial data
        self.datacollector.collect(self)

    #LAYERS OF THE GRID--------------------------------------
    def initialize_layers(self):
        """Initialize the grid layers for buildings, parking spots, and traffic monitoring."""
        buildingLayer = mesa.space.PropertyLayer("building", self.width, self.height, np.int64(0), np.int64)
        parkingsLayer = mesa.space.PropertyLayer("parking", self.width, self.height, np.int64(0), np.int64)
        trafficMonitoringLayer = mesa.space.PropertyLayer("traffic_monitoring", self.width, self.height, np.int64(0), np.int64)

        self.set_building_cells(buildingLayer)
        self.set_parking_cells(parkingsLayer)
        self.set_traffic_monitoring_cells(trafficMonitoringLayer)

        self.grid = mesa.space.MultiGrid(self.width, self.height, True, (buildingLayer, parkingsLayer, trafficMonitoringLayer))
    #INITIALIZER METHODS----------------------------------------------------------------------------------
    # Initialize allowed directions for each cell in the grid
    def initialize_directions(self, coords):
        for x in range(self.width):
            for y in range(self.height):
                self.directions[(x, y)] = {
                    "left": False,
                    "right": False,
                    "up": False,
                    "down": False,
                    "down_left": False,
                    "down_right": False,
                    "up_left": False,
                    "up_right": False,
                    "traffic_left": False,
                    "traffic_right": False,
                    "traffic_up": False,
                    "traffic_down": False,
                }

        # Set specific directions for each list
        for coord in coords["left_coords"]:
            if coord in self.directions:
                self.directions[coord]["left"] = True

        for coord in coords["right_coords"]:
            if coord in self.directions:
                self.directions[coord]["right"] = True

        for coord in coords["up_coords"]:
            if coord in self.directions:
                self.directions[coord]["up"] = True

        for coord in coords["down_coords"]:
            if coord in self.directions:
                self.directions[coord]["down"] = True

        for coord in coords["down_left_coords"]:
            if coord in self.directions:
                self.directions[coord]["down_left"] = True

        for coord in coords["down_right_coords"]:
            if coord in self.directions:
                self.directions[coord]["down_right"] = True

        for coord in coords["up_left_coords"]:
            if coord in self.directions:
                self.directions[coord]["up_left"] = True

        for coord in coords["up_right_coords"]:
            if coord in self.directions:
                self.directions[coord]["up_right"] = True

    def create_CarAgents_no_target(self):
        """Create car agents without a target parking spot."""
        number_of_cars = 50
        all_coords = [(x, y) for x in range(self.width) for y in range(self.height)]
        available_coords = [coord for coord in all_coords if coord not in self.parkings_coords and coord not in self.buildings_coords]

        if len(available_coords) < number_of_cars:
            raise ValueError("Not enough available coordinates to place all cars.")

        for _ in range(number_of_cars):
            spawn_position = random.choice(available_coords)
            available_coords.remove(spawn_position)
            agent = CarAgent(self, spawn_position, None)
            self.grid.place_agent(agent, spawn_position)

     # Create agents and place them on the grid
    def create_CarAgents(self):
        """Create car agents and place them on the grid."""
        used_parking_spots = set()

        for _ in range(self.num_agents):
            available_spawn_spots = [spot for spot in self.ParkingSpots.values() if spot not in used_parking_spots]

            if not available_spawn_spots:
                print("No available spots for spawn")
                break

            spawn = random.choice(available_spawn_spots)
            used_parking_spots.add(spawn)

            available_target_spots = [spot for spot in self.ParkingSpots.values() if spot != spawn and spot not in used_parking_spots]

            if not available_target_spots:
                print("No available spots for target parking")
                break

            target_parking_spot = random.choice(available_target_spots)
            used_parking_spots.add(target_parking_spot)

            print(f"Spawn: ({spawn}), Target: ({target_parking_spot})")

            agent = CarAgent(self, spawn, target_parking_spot)
            self.grid.place_agent(agent, spawn)

    def create_CarAgents_no_target(self):
        """Create car agents without a target parking spot."""
        number_of_cars = 50
        all_coords = [(x, y) for x in range(self.width) for y in range(self.height)]
        available_coords = [
            coord
            for coord in all_coords
            if coord not in self.parkings_coords and coord not in self.buildings_coords
        ]
        if len(available_coords) < number_of_cars:
            raise ValueError("Not enough available coordinates to place all cars.")
        for _ in range(number_of_cars):
            spawn_position = random.choice(available_coords)
            available_coords.remove(spawn_position)
            agent = CarAgent(self, spawn_position, None)
            self.grid.place_agent(agent, spawn_position)


    # Place traffic light agents on the grid
# Place traffic light agents on the grid
    def place_TrafficLight_agents(self):
        for idx, traffic_light_info in enumerate(self.traffic_light_coords):
            traffic_light_id = idx # Unique ID for each traffic light
            positions = traffic_light_info[:2]  # Assuming the positions are the first two values
            initial_state = traffic_light_info[2]  # The initial state is the third value
            monitored_coords = traffic_light_info[3]
            #print(f"Monitored positions: {monitored_coords}")

            # Create the traffic light agent for each position
            for pos in positions:
                # Ensure the position is passed when creating the agent
                sema_agent = TrafficLightAgent(
                    traffic_light_id, initial_state, self,monitored_positions=monitored_coords

                )  # Assign the ID, state, model, and position

                # Place the agent on the grid
                self.grid.place_agent(sema_agent, pos)

                # Print information about the placed agent
                #print(f"Placed TrafficLightAgent: ID={sema_agent.unique_id}, State={sema_agent.state}, Position={sema_agent.pos}")
                
        
                
                
    #GETTER METHODS----------------------------------------------------------------------------------
    # Fetch the direction info for a specific cell
    def get_cell_directions(self, pos):    
        return self.directions.get(pos, None)

    # Create a global map of the current state of the simulation
    def get_global_map(self):
        """
        Generate a global map of the current state of the simulation in the specified JSON format.
        """
        # Inicializar el mapa global
        self.global_map = {
            "Cars": [],
            "Traffic_Lights": {}
        }

        # Recopilar agentes
        for contents, (x, y) in self.grid.coord_iter():
            for agent in contents:
                if isinstance(agent, CarAgent):
                    # Agregar las coordenadas del agente CarAgent al formato deseado
                    self.global_map["Cars"].append({"x": x, "y": y})
                elif isinstance(agent, TrafficLightAgent):
                    # Agregar datos del semáforo al formato deseado
                    self.global_map["Traffic_Lights"][f"sema_{agent.unique_id}"] = agent.state

        # Debug: Imprimir el mapa global
        print(self.global_map)

        # Devolver el mapa global en el formato solicitado
        return self.global_map

    
        
    # LAYER METHODS----------------------------------------------------------------------------------
    def set_building_cells(self, buildingLayer):
        """Set the value of cells to indicate buildings."""
        for coord in self.buildings_coords:
            buildingLayer.set_cell(coord, 1)

    def set_parking_cells(self, parkingsLayer):
        """Set the value of cells to indicate parking spots."""
        for coord in self.parkings_coords:
            parkingsLayer.set_cell(coord, 1)

    def set_traffic_monitoring_cells(self, trafficMonitoringLayer):
        """Set the value of cells to indicate traffic monitoring spots."""
        all_monitored_coords = []
        for traffic_light in self.traffic_light_coords:
            # Access monitored coordinates assuming they are the last element in each list
            monitored_coords = traffic_light[-1]
            all_monitored_coords.extend(monitored_coords)

        for coord in all_monitored_coords:
            trafficMonitoringLayer.set_cell(coord, 1)

    #TRAFFIC MOTHERATION LAYERS----------------------------------------------------------------------------------
    def traffic_in_area(self, area ):
        count = 0
        for pos in area:
            for agent in self.grid.get_cell_list_contents(pos):
                if isinstance(agent, CarAgent):
                    count += 1
        if count > 10:
            return True
        else:
            return False
        
    def modify_street_with_traffic(self):
        for key, value in coords["monitoring_coords"].items():
            pos = value["pos"]
            area = value["area"]
            direction = value["direction"]

            if self.traffic_in_area(area):
                self.directions[pos][direction] = False
            else:
                self.directions[pos][direction] = True

    # STEP METHOD----------------------------------------------------------------------------------
    # Execute one step of the model, shuffle agents, and collect data
    def step(self):
        # Shuffle and execute the step method for all agents
        self.agents.shuffle_do("step")
        # Collect data for the current step
        self.datacollector.collect(self)
        # Create a global map of the current state
        self.get_global_map()






## Car Agent
Main points of the agent's functionality:

- Initialization:
Each agent has a unique ID and is created with the model. 
It is randomly positioned on the model's grid.
- Movement:
The main movement method is `move()`.
This method is called at each model step `step()`.
- Directions:
A dictionary of directions is used: "up," "down," "left," "right".
- Choosing a random direction:
A random direction is selected from those allowed in the current cell.
- Checking for an empty space:
Before moving, it checks if the new space is empty.

**Basic Configuration:**
- Defines the width and height of the grid (default is 24x24). 
Sets the number of agents (default is 10).
- Path Customization:
Allows defining custom paths using lists of coordinates (x, y) for each direction (up, down, left, right).
- Model Initialization: 
Creates a two-dimensional grid (MultiGrid). 
Initializes the scheduler to manage model steps.
- Agent Creation: 
Randomly places agents on the grid.
- Direction Initialization: 
Uses the initialize_directions() method to set up allowed directions in each cell based on the provided lists.
- Model Steps: The step() method is called at each step to update the state of all agents.

In [10]:
class CarAgent(mesa.Agent):
    def __init__(self, model, spawn_position, target_parking_spot = None):
        super().__init__(model)
        self.active = True
        self.parking_spots = [coord for coord in model.parkings_coords if coord != spawn_position]
        self.distance_travelled = 0
        self.target_parking_spot = target_parking_spot

    def check_semaphore(self, current_position): #for future implementation, check if the agent is a car
        agents_at_position = self.model.grid.get_cell_list_contents([current_position])
        semaphore_agent = None
        for agent in agents_at_position:
            if agent.__class__ == TrafficLightAgent:
                semaphore_agent = agent
        if semaphore_agent is None:
            return True
        else:
            if semaphore_agent.state == 1:
                print(f"Semaphore at {current_position} is red; agent cannot move.")
                return False
            elif semaphore_agent.state == 2:
                print(f"Semaphore at {current_position} is green; agent can move.")
                return True

    def check_agent(self, new_position):
        agents_at_position = self.model.grid.get_cell_list_contents([new_position])
        car_agent = None
        for agent in agents_at_position:
            if agent.__class__ == CarAgent:
                car_agent = agent
        if car_agent is None: 
            return True
        else:
            return False

    def move(self):
        # Get current position and allowed directions
        current_position = self.pos
        possible_current_directions = self.model.get_cell_directions(current_position)
        
        # Check for any available directions in this cell
        if not possible_current_directions:
            print(f"No directions available for agent at position {current_position}")
            return

        # Filter allowed directions and select one randomly
        possible_directions = self.get_possible_directions(possible_current_directions)
        if not possible_directions:
            print(f"No movement options for agent at position {current_position}")
            return
        
        # Checks if it can move acccording to the sempahore
        if not self.check_semaphore(current_position):
            return 
        
        direction = self.choose_direction(possible_directions)
        new_position = self.calculate_new_position(direction)

        # Logic to not allow the agent to move to a parking spot that is not its target
        while new_position in self.model.parkings_coords and new_position != self.target_parking_spot:
            direction = self.choose_direction(possible_directions)
            new_position = self.calculate_new_position(direction)

        if not self.check_agent(new_position):
            return
 
        self.update_position(new_position)

    def get_possible_directions(self, possible_current_directions):
        return [direction for direction, allowed in possible_current_directions.items() if allowed]

    def choose_direction(self, possible_directions):
        direction_weights = {
            "up": 1,
            "down": 1,
            "left": 1,
            "right": 1,
            "down_left": 0.1,
            "down_right": 0.1,
            "up_left": 0.1,
            "up_right": 0.1
        }
        weights = [direction_weights[direction] for direction in possible_directions]
        return random.choices(possible_directions, weights=weights, k=1)[0]
    
    def calculate_new_position(self, direction):
        direction_map = {
            "up": (0, 1),
            "down": (0, -1),
            "left": (-1, 0),
            "right": (1, 0),
            "down_left": (-1, -1),
            "down_right": (1, -1),
            "up_left": (-1, 1),
            "up_right": (1, 1),
        }
        dx, dy = direction_map[direction]
        return (self.pos[0] + dx, self.pos[1] + dy)
    
    def update_position(self, new_position):
        self.model.grid.move_agent(self, new_position)
        self.distance_travelled += 1
        self.pos = new_position


    def move_to_target(self):
        # Moverse hasta alcanzar el destino
        while self.active:
            if self.pos in self.parking_spots and self.distance_travelled > 0:
                self.active = False
                break
            else:
                moved = self.move()
                if not moved:  # Si no puede moverse, detener el bucle
                    break            
            
    def step(self):
        self.move_to_target()

## Traffic Light Agent

In [11]:
class TrafficLightAgent(mesa.Agent):
    def __init__(self, unique_id, state, model, monitored_positions):
        super().__init__(model)
        self.unique_id = unique_id
        self.state = state  
        self.time_counter = 0
        self.monitored_positions = monitored_positions
        self.neighbor_siblings = []  
        self.neighbor_opposites = []  

    def update_neighbors(self):
        """Actualiza las listas de semáforos hermanos y opuestos."""
        neighbors = self.model.grid.get_neighbors(self.pos, moore=True, include_center=False)

        self.neighbor_siblings = [
            agent for agent in neighbors if isinstance(agent, TrafficLightAgent) and agent.unique_id == self.unique_id
        ]
        self.neighbor_opposites = [
            agent for agent in neighbors if isinstance(agent, TrafficLightAgent) and agent.unique_id != self.unique_id
        ]

    def change_state(self, state):
        """Cambia el estado del semáforo automáticamente."""
        self.state = state

    def cars_in_monitored_area(self):
        """Revisa si hay autos en las posiciones monitoreadas."""
        in_area_cars = 0
        for pos in self.monitored_positions:
            agents = self.model.grid.get_cell_list_contents([pos])
            for agent in agents:
                if isinstance(agent, CarAgent):
                    in_area_cars += 1
        return in_area_cars


    def compare_traffic_with_neighbors(self):
        """Compara el tráfico en su área monitoreada con los vecinos y ajusta estados."""
        current_traffic = self.cars_in_monitored_area()
        
        # Imprimir información del agente actual
        print(f"Traffic Light {self.unique_id} at {self.pos} with current state {self.state}")

        # Imprimir información de vecinos hermanos
        print(f"  Sibling neighbors of Traffic Light {self.unique_id}:")
        for sibling in self.neighbor_siblings:
            sibling_traffic = sibling.cars_in_monitored_area()
            print(f"    Sibling {sibling.unique_id} at {sibling.pos} with state {sibling.state} and traffic {sibling_traffic}")

        # Imprimir información de vecinos opuestos
        print(f"  Opposite neighbors of Traffic Light {self.unique_id}:")
        for opposite in self.neighbor_opposites:
            opposite_traffic = opposite.cars_in_monitored_area()
            print(f"    Opposite {opposite.unique_id} at {opposite.pos} with state {opposite.state} and traffic {opposite_traffic}")

        # Cambiar el estado basado en el tráfico
        if current_traffic == 0:
            self.change_state(3)
            for sibling in self.neighbor_siblings:
                sibling.change_state(3)
            return

        tempSelf = None  
        for opposite in self.neighbor_opposites:
            opposite_traffic = opposite.cars_in_monitored_area()
            if current_traffic > opposite_traffic:
                self.state = 2
                if opposite_traffic == 0:
                    opposite_traffic = 3
                else :
                    opposite.state = 1

                tempSelf = opposite  
                for sibling in self.neighbor_siblings:
                    sibling.state = self.state


        # Obtener vecinos de tempSelf
        if tempSelf:
            print(f"TempSelf is set to Opposite Traffic Light {tempSelf.unique_id} at {tempSelf.pos} with state {tempSelf.state}")
            tempSelf_neighbors = self.model.grid.get_neighbors(tempSelf.pos, moore=True, include_center=False)
            tempSelf_traffic_lights = [
                agent for agent in tempSelf_neighbors if isinstance(agent, TrafficLightAgent)
            ]

            # Imprimir vecinos de tempSelf
            print(f"  Neighbors of TempSelf (Traffic Light {tempSelf.unique_id}):")
            for neighbor in tempSelf_traffic_lights:
                neighbor_traffic = neighbor.cars_in_monitored_area()
                print(f"    Neighbor {neighbor.unique_id} at {neighbor.pos} with state {neighbor.state} and traffic {neighbor_traffic}")
                if tempSelf.unique_id == neighbor.unique_id:
                    neighbor.state = tempSelf.state




    def step(self):
        """Actualizar el estado en cada paso."""
        self.time_counter += 1
        self.update_neighbors()  # Asegurarse de que las listas de vecinos estén actualizadas.
        self.compare_traffic_with_neighbors()


## Model Deployment 

In [12]:
# Crear una instancia de TrafficModel
model = TrafficModel(24, 24, 5, coords, buildings_coords, parking_spots, traffic_light_coords)

# Crear una matriz para contar los agentes de tipo CarAgent
agent_counts = np.zeros((model.grid.width, model.grid.height))

# Imprimir las coordenadas de los agentes
print("Coordenadas de los agentes:")
for agent in model.agents:  # Itera sobre todos los agentes en el modelo
    if isinstance(agent, CarAgent):  # Verifica si es un CarAgent
        print(f"Agente en posición: {agent.pos}")



Spawn: ((8, 15)), Target: ((4, 12))
Spawn: ((17, 21)), Target: ((4, 3))
Spawn: ((10, 19)), Target: ((10, 12))
Spawn: ((17, 4)), Target: ((20, 18))
Spawn: ((17, 6)), Target: ((2, 14))
Coordenadas de los agentes:
Agente en posición: (8, 15)
Agente en posición: (17, 21)
Agente en posición: (10, 19)
Agente en posición: (17, 4)
Agente en posición: (17, 6)
Agente en posición: (13, 15)
Agente en posición: (6, 9)
Agente en posición: (1, 18)
Agente en posición: (23, 10)
Agente en posición: (12, 19)
Agente en posición: (17, 9)
Agente en posición: (5, 10)
Agente en posición: (8, 1)
Agente en posición: (23, 0)
Agente en posición: (20, 8)
Agente en posición: (7, 23)
Agente en posición: (23, 22)
Agente en posición: (15, 9)
Agente en posición: (17, 0)
Agente en posición: (17, 17)
Agente en posición: (0, 11)
Agente en posición: (4, 9)
Agente en posición: (19, 6)
Agente en posición: (1, 16)
Agente en posición: (9, 1)
Agente en posición: (0, 16)
Agente en posición: (6, 19)
Agente en posición: (22, 19)
A

C:\Users\ricky\AppData\Local\Temp\ipykernel_3048\3990760335.py:70: FutureWarning: The new PropertyLayer and _PropertyGrid classes experimental. It may be changed or removed in any and all future releases, including patch releases.
We would love to hear what you think about this new feature. If you have any thoughts, share them with us here: https://github.com/projectmesa/mesa/discussions/1932
  buildingLayer = mesa.space.PropertyLayer("building", self.width, self.height, np.int64(0), np.int64)


In [13]:
for i in range(1):  # Eje    cutar el modelo por 10 pasos
  model.step()

Traffic Light 2 at (7, 16) with current state 3
  Sibling neighbors of Traffic Light 2:
    Sibling 2 at (6, 16) with state 3 and traffic 1
  Opposite neighbors of Traffic Light 2:
    Opposite 3 at (8, 17) with state 3 and traffic 0
TempSelf is set to Opposite Traffic Light 3 at (8, 17) with state 3
  Neighbors of TempSelf (Traffic Light 3):
    Neighbor 2 at (7, 16) with state 2 and traffic 1
    Neighbor 3 at (8, 18) with state 3 and traffic 0
Traffic Light 6 at (5, 1) with current state 3
  Sibling neighbors of Traffic Light 6:
    Sibling 6 at (5, 0) with state 3 and traffic 1
  Opposite neighbors of Traffic Light 6:
    Opposite 7 at (6, 2) with state 3 and traffic 1
Traffic Light 4 at (0, 6) with current state 3
  Sibling neighbors of Traffic Light 4:
    Sibling 4 at (1, 6) with state 3 and traffic 1
  Opposite neighbors of Traffic Light 4:
Traffic Light 1 at (8, 23) with current state 3
  Sibling neighbors of Traffic Light 1:
    Sibling 1 at (8, 22) with state 3 and traffic 0

# Visualization 1

In [14]:
#%pip install solara

In [15]:
import solara


Define the agent portrayal function

In [16]:
def agent_portrayal(agent):
    size = 20
    color = "tab:red"
    shape = "circle"
    
    # Verificar el tipo de agente
    if isinstance(agent, CarAgent):
        size = 50
        color = "tab:blue"
        shape = "circle"
    elif isinstance(agent, TrafficLightAgent):
        size = 20
        shape = "circle"
        if agent.state == 1:
            color = "tab:red"
        elif agent.state == 2:
            color = "tab:green"
        elif agent.state == 3:  
            color = "yellow"  
    return {"size": size, "color": color, "shape": shape}




 Define the model parameters

In [17]:
model_params = {
    "num_agents": {
        "type": "SliderInt",
        "value": 10,
        "label": "Number of agents:",
    },
    "width": 24,
    "height": 24,
}

Create the initial model instance

In [18]:
initial_model = TrafficModel(
    num_agents=10,
    width=24,
    height=24,
    coords = coords,
    buildings_coords=buildings_coords,
    parking_coords=parking_spots,
    traffic_light_coords=traffic_light_coords
)



Spawn: ((10, 19)), Target: ((5, 17))
Spawn: ((4, 12)), Target: ((17, 21))
Spawn: ((20, 15)), Target: ((17, 4))
Spawn: ((4, 3)), Target: ((9, 2))
Spawn: ((10, 7)), Target: ((20, 4))
Spawn: ((3, 21)), Target: ((3, 6))
Spawn: ((8, 15)), Target: ((2, 14))
Spawn: ((20, 18)), Target: ((10, 12))
No available spots for target parking


Create the visualization components

In [19]:
propertylayer_portrayal={"building": {"color":"blue","colorbar":False}, "parking":{"color":"yellow","colorbar":False}, "traffic_monitoring":{"color":"gray","colorbar":False}}
SpaceGraph = make_space_component(agent_portrayal, propertylayer_portrayal=propertylayer_portrayal)

Create the SolaraViz page

In [ ]:
page = SolaraViz(
    initial_model,
    components=[SpaceGraph],
    model_params=model_params,
    name="Traffic Simulation",
)
page

d:\Aplicaciones\Python\Lib\site-packages\mesa\visualization\mpl_space_drawing.py:272: UserWarning: the following fields are not used in agent portrayal and thus ignored: shape.
  arguments = collect_agent_data(space, agent_portrayal, size=s_default)
d:\Aplicaciones\Python\Lib\site-packages\mesa\visualization\mpl_space_drawing.py:272: UserWarning: the following fields are not used in agent portrayal and thus ignored: shape.
  arguments = collect_agent_data(space, agent_portrayal, size=s_default)


Cannot show ipywidgets in text

Traffic Light 8 at (17, 9) with current state 3
  Sibling neighbors of Traffic Light 8:
    Sibling 8 at (17, 8) with state 3 and traffic 0
  Opposite neighbors of Traffic Light 8:
Traffic Light 3 at (8, 18) with current state 3
  Sibling neighbors of Traffic Light 3:
    Sibling 3 at (8, 17) with state 3 and traffic 0
  Opposite neighbors of Traffic Light 3:
Traffic Light 2 at (6, 16) with current state 3
  Sibling neighbors of Traffic Light 2:
    Sibling 2 at (7, 16) with state 3 and traffic 3
  Opposite neighbors of Traffic Light 2:
Traffic Light 0 at (6, 21) with current state 3
  Sibling neighbors of Traffic Light 0:
    Sibling 0 at (7, 21) with state 3 and traffic 1
  Opposite neighbors of Traffic Light 0:
Traffic Light 6 at (5, 1) with current state 3
  Sibling neighbors of Traffic Light 6:
    Sibling 6 at (5, 0) with state 3 and traffic 1
  Opposite neighbors of Traffic Light 6:
    Opposite 7 at (6, 2) with state 3 and traffic 0
TempSelf is set to Opposite Traffic Light 7 a